Create the skeleton of the network

In [2]:
# create a SimpleCNN class that inherits pytorch.nn.module and implement a vanilla CNN

# file vanilla_cnn_2.py
# author: Kazi Mahbub Mutakabbir
# Email:kazi.mahbub@tum.de
# date 17-10-2019

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
import numpy as np
from PIL import Image
from torch.autograd import Variable
from progress.bar import Bar
import matplotlib.pyplot as plt
import csv


In [3]:
class SimpleCNN(nn.Module):

    # batch shape for input: (3, 32, 32), 3 = RGB channels

    def __init__(self):

        super(SimpleCNN, self).__init__()
        # super().__init__()

        ### first convolutional layer ###
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0)
        # batch normalization
        self.batch_norm_1 = nn.BatchNorm2d(16)
        # reLU activation
        self.relu = nn.ReLU()
        # pooling layerZ
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # kernel size after 1st conv layer: 111*111*16

        ### 2nd convolutional layer ###
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)
        # batch normalization layer
        self.batch_norm_2 = nn.BatchNorm2d(32)
        # pooling layer
        # 1 padding added to get whole value of kernel dimension
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        # kernel size after 2nd conv layer: 55*55*32

        ### 3rd convolutional layer ###
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        # batch normalization layer
        self.batch_norm_3 = nn.BatchNorm2d(64)
        # pooling layer
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # kernel size after 3rd conv layer: 26*26*64

        ### 4th convolutional layer ###
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0)
        # batch normalization layer
        self.batch_norm_4 = nn.BatchNorm2d(128)
        # pooling layer
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # kernel size after 4th convolutional layer: 12*12*128

        ### 5th convolutional layer ###
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0)
        # batch normalization layer
        self.batch_norm_5 = nn.BatchNorm2d(256)
        # pooling layer
        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # kernel size after 5th convolutional layer: 5*5*256

        # fully connected layer 1
        self.fc1 = nn.Linear(in_features=256 * 5 * 5, out_features=4096)
        self.batch_norm_6 = nn.BatchNorm1d(4096)
        # add a dropout to reduce overfitting
        #self.dropout = nn.Dropout(p=0.3)

        # fully connected layer 2
        self.fc2 = nn.Linear(in_features=4096, out_features=1024)
        # add a dropout to reduce overfitting
        self.dropout = nn.Dropout(p=0.3)

        # fully connected layer 3
        self.fc3 = nn.Linear(1024, 256)
        # add a dropout
        self.dropout = nn.Dropout(p=0.5)

        # output layer
        self.fc4 = nn.Linear(256,2)


    def forward(self, x):
        # compute the activation of the first convolutional layer
        out = self.conv1(x)
        out = self.batch_norm_1(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        
        # compute the activation of the second convolutional layer
        out = self.conv2(out)
        out = self.batch_norm_2(out)
        out = self.relu(out)
        out = self.maxpool2(out)
        
        # compute the activation of the third convolutional layer
        out = self.conv3(out)
        out = self.batch_norm_3(out)
        out = self.relu(out)
        out = self.maxpool3(out)

        # compute the activation of the fourth convolutional layer
        out = self.conv4(out)
        out = self.batch_norm_4(out)
        out = self.relu(out)
        out = self.maxpool4(out)

        # compute the activation of the fourth convolutional layer
        out = self.conv5(out)
        out = self.batch_norm_5(out)
        out = self.relu(out)
        out = self.maxpool5(out)
        # print("Shape after conv: " + str(out.shape))
        # Reshape data for the input layer of the net
        # Recall that the -1 infers this dimension

        out = out.view(-1, 256 * 5 * 5)

        # last conv layer -> 1st fc layer
        out = self.fc1(out)
        out = self.batch_norm_6(out)
        out = self.relu(out)
        #out = self.dropout(out)

        # 1st fc layer -> 2nd fc layer
        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)

        # 2nd fc layer -> 3rd fc layer
        out = self.fc3(out)
        out = self.relu(out)
        out = self.dropout(out)

        # 3rd fc layer -> output layer
        out = self.fc4(out)
        #out = self.relu(out)
        #out = self.dropout(out)

        return (out)
    

Loading the model and restoring state

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = SimpleCNN()
state_dict = torch.load("simpleCNN_2019-11-04 04_44_47.pt")
#print(state_dict)
model.load_state_dict(state_dict)
#print(model)
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm_3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm_4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [5]:
dataset = os.getcwd() + "\\processedscreenshots"
transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(
                                        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)
                                        )])
# test_data = datasets.ImageFolder(root=dataset, transform=transform)
# test_loader  = data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)
# correct = 0
# total = 0


In [6]:
def predict_image(image):
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    print(output)
    index = output.data.cpu().numpy().argmax()
    return index

In [7]:
filenames = os.listdir(dataset)
# print(filenames)
classes = ['neg', 'pos']
to_pil = transforms.ToPILImage()
model.cuda()
output_file = "output.csv"
file_count = len(filenames)
bar = Bar('Processing', max=20)

for i in range(file_count):
    #print(filenames[i])
    image_to_be_predicted = Image.open(dataset+"\\"+filenames[i])
    #image_to_be_predicted = dataset+"\\"+filenames[i]
    img_t = transform(image_to_be_predicted)
    img_t = to_pil(img_t)
    index = predict_image(img_t)
    data = [filenames[i], classes[index]]
    #print(index)
    #print("class: " + classes[index])
    if (i==0):
        with open(output_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)
    else:
        with open(output_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)
    bar.next()

bar.finish()
print("task finished.")

tensor([[-0.1275,  0.0227]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.6070, -2.2763]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.1081, -0.2840]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3167, -3.3436]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.3012, -1.9070]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.4965, -3.4874]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.2839, -0.5027]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.1926, -0.3111]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.7707, -1.0626]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1954, -3.2311]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.3428, -1.9621]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.7461, -2.6303]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.0147, -0.0847]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.2022, -0.3838]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

tensor([[ 0.4271, -0.7671]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.2648, -2.0356]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.2934, -0.4322]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9259, -2.7886]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.5848, -2.3279]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.7480, -1.0575]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 0.0765, -0.2286]], device='cuda:0', grad_fn=<AddmmBackward>)
task finished.
